In [5]:
!!pip install beautifulsoup4

['Requirement already satisfied: beautifulsoup4 in /Users/cglacet/.pyenv/versions/3.8.0/lib/python3.8/site-packages (4.9.1)',
 'Requirement already satisfied: soupsieve>1.2 in /Users/cglacet/.pyenv/versions/3.8.0/lib/python3.8/site-packages (from beautifulsoup4) (2.0.1)']

In [6]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd

ROOT_URL = 'https://rural-camping.com/france/'

In [9]:
def places_pages_urls():
    # 1) On récupère le code HTML (juste du texte) et on donne ça à BeautifulSoup :
    response = requests.get(ROOT_URL + 'camping-map-france.htm')
    html = BeautifulSoup(response.text)

    # 2) On cherche les éléments dans la page, par exemple on peut 
    #    dire qu'on cherche les 'table' HTML qui appartiennent à 'ment'.
    # (Les classes en HTML servent surtout à définir des groupes d'éléments 
    # pour appliquer un style au groupe.)
    table = html.find('table', class_='ment')
    for link in table.find_all('a'):
        # Dès qu'on trouve un élément (lien dans ce cas), on peut accéder à ses propriétés). Nous on veut juste l'URL référencée par le lien (href) :
        yield ROOT_URL + link.get('href')

Avant d'écrire la suite on peut déjà tester que ça donne bien une liste d'URLs: 

In [10]:
print(list(places_pages_urls()))

['https://rural-camping.com/france/index.htm#1', 'https://rural-camping.com/france/brittany-normandy.htm', 'https://rural-camping.com/france/northeast.htm', 'https://rural-camping.com/france/loire-burgundy.htm', 'https://rural-camping.com/france/limousin-auvergne.htm', 'https://rural-camping.com/france/southwest-france.htm', 'https://rural-camping.com/france/alps-jura.htm', 'https://rural-camping.com/france/south-france-midi.htm', 'https://rural-camping.com/france/midi-pyrenees.htm', 'https://rural-camping.com/france/south-france-midi.htm']


In [13]:
# Maintenant on va utiliser cette fonction pour trouver les liens vers les locations : 
def place_info():
    for place_page_url in places_pages_urls():
        response = requests.get(place_page_url)
        print("Request for page", place_page_url)
        html = BeautifulSoup(response.text)
        # On cherche des items (li) dans des listes (ul),
        # et dans chaque item on cherche un lien 
        for ul in html.find_all('ul'):
            for li in ul.find_all('li'):
                link = li.find('a')
                # Si on a trouvé un lien on renvoie le lien et le texte autour 
                # (celui de l'item dans la liste)
                if link:
                    yield [li.get_text(), link.get('href')]

In [15]:
pd.DataFrame(list(place_info()), columns=['text', 'link'])

Request for page https://rural-camping.com/france/index.htm#1
Request for page https://rural-camping.com/france/brittany-normandy.htm
Request for page https://rural-camping.com/france/northeast.htm
Request for page https://rural-camping.com/france/loire-burgundy.htm
Request for page https://rural-camping.com/france/limousin-auvergne.htm
Request for page https://rural-camping.com/france/southwest-france.htm
Request for page https://rural-camping.com/france/alps-jura.htm
Request for page https://rural-camping.com/france/south-france-midi.htm
Request for page https://rural-camping.com/france/midi-pyrenees.htm
Request for page https://rural-camping.com/france/south-france-midi.htm


,text,link
0,Campsite in Camors\nâ¦ (56\r\nMorbihan) In ...,https://www.lepetitbois-camping.bzh/
1,Camping in\r\ncentral Brittany\nâ¦ (56\r\nMo...,https://www.camping-des-cerisiers.fr
2,Camping in Morbihan\nâ¦ (56\r\nMorbihan) In ...,http://en.camping-pontcalleck.fr/
3,Le\r\nBalcon de la Baie\r\n- â¦\n(35 Ille et\...,http://www.lebalcondelabaie.com/
4,La\r\nFerme de Croas Men\r\n- (29 -\r\nFinist...,http://ferme-de-croasmen.com/
...,...,...
88,Eastern\r\nPyrenees ~\r\n(66 Eastern Pyrenees)...,http://www.solane.info/
89,Campsite\r\nin the Cevennes ~\r\n( 30 Gard) A ...,http://campinglabahou-anduze.c.la/
90,"Campsite\r\nin\r\nrural Languedoc ~\r\nRustic,...",http://www.goudal.com/en
91,Campsite\r\nat the foot of the Cevennes (30 Ga...,https://www.camping-mascauvy.com/
